## Monte Carlo Simulation to Forecast Price of one Stock

Monte Carlo can be used to simulate future stick prices. This simulation strategy makes use of the Law of Large Numbers that makes prediction on the basis of historical data

In [1]:
import pandas as pd 
import numpy as  np 
from pandas_datareader import data as wb 
import matplotlib.pyplot as plt 
import seaborn as sns 
from scipy.stats import norm 
import yfinance as yf 
from datetime import datetime, timedelta 
import seaborn as sns 

import warnings 
warnings.filterwarnings('ignore') 

In [ ]:
start_date = '2020-01-01' 
ticker = 'GOOG'

In [ ]:
data = yf.download(ticker,start=start_date) 
data.head() 

#### Plot Closing Price

In [ ]:
plt.figure(figsize=(20,7)) 
plt.plot(data['Close'],label='Close Price') 
plt.title(f'Close Price of {ticker} since {start_date}') 
plt.xlabel('Date') 
plt.ylabel('Close Price ($)') 
plt.style.use('seaborn-v0_8-darkgrid') 
plt.legend(loc='best')  
plt.grid(True) 
plt.show()

### Daily Log Returns 

* For small prices movements, simple returns and log returns are quite similar. But, for large price movements, log returns can provide more accurate and realistic representation of returns 
* Log returns are additive in nature making it useful for analytical and computational analysis 
* Log returns also capture skewness in financial returns better than mean returns 

In [ ]:
def log_returns(data):
    log_returns = np.log(1+data['Close'].pct_change()) 
    log_returns = log_returns[1:] 
    return log_returns  

#### Plotting Log-Returns

In [ ]:
log_return = log_returns(data) 
plt.figure(figsize=(20,7)) 
plt.plot(log_return,label='Log Returns') 
plt.title(f'Log Returns of {ticker} since {start_date}') 
plt.xlabel('Date') 
plt.ylabel('Log Returns') 
plt.legend(loc='best') 
plt.grid(True) 
plt.show() 

### Calculating Volatility

Volatility is the standard deviation of daily returns

In [ ]:
def volatility_calc(log_rtn):
    daily_volatility = np.std(log_rtn) 
    return daily_volatility 

print(volatility_calc(log_return)) 

## Running Simulations

#### Variable Selection
* More the number of simulations, the more accurate will be the data. However, we have take in consideration of the hardware used for this purpose
* Choose the number of days to be simulated. 252 is chosen as it is the number of trading days in a year

In [ ]:
num_sims = 10000
num_days = 100 

Getting last closing price of data 

In [ ]:
last_price = data['Close'].iloc[-1] 

#### Simulation Function

In [ ]:
def monte_carlo_sim(num_sims,num_days,last_price,log_return):
    daily_vol = volatility_calc(log_return) 

    all_simulations = [] 

    for x in range(num_sims):
        price_series = [last_price] 

        for y in range(1,num_days):
            price = price_series[-1]*(1+np.random.normal(0,daily_vol)) 
            price_series.append(price) 
        all_simulations.append(price_series) 

    sim_df = pd.DataFrame(all_simulations).transpose() 
    return sim_df        

In [ ]:
simulation_df = monte_carlo_sim(num_sims,num_days,last_price,log_return) 
simulation_df.head() 

#### Plot Monte Carlo

In [ ]:
simulation_df.plot(legend=False) 
plt.title(f'Monte Carlo Simulation for {ticker}') 
plt.axhline(y=last_price,color='black',linestyle='-') 
plt.xlabel('Day') 
plt.ylabel('Price') 
plt.show() 

### Testing

In [ ]:
print('Start date : ',data.iloc[0].name)
print('Last date : ',data.iloc[-1].name) 

Lets try to predict the price of stock 2 days later

In [ ]:
day_1_price = simulation_df.iloc[0] 
day_2_price = simulation_df.iloc[1] 

# Calculate 95% confidence interval 
lower_bound_95 = np.percentile(day_2_price, 2.5) 
upper_bound_95 = np.percentile(day_2_price, 97.5) 

mean_price = np.mean(day_2_price) 

# Calculate 98% Confidence Interval
lower_bound_98 = np.percentile(day_2_price,1) 
upper_bound_98 = np.percentile(day_2_price,99) 

print(f'Expected price 2 days later : {mean_price:.4f}') 
print(f'95% Confidence Interval for price 2 days later : ({lower_bound_95:.4f},{upper_bound_95:.4f})')
print(f'98% Confidence Interval for price 2 days later : ({lower_bound_98:.4f},{upper_bound_98:.4f})')

### Defining Monte Carlo Simulation Class

In [8]:
class MonteCarlo():
    def __init__(self,ticker,start_date,end_date,num_days,num_sims):
        self.ticker = ticker 
        self.start_date = start_date
        self.end_date = end_date
        self.num_days = num_days 
        self.num_sims = num_sims 

    def fetch_data(self):
        self.data = pd.DataFrame() 
        self.data = yf.download(self.ticker,start=self.start_date,end=self.end_date)

    def calc_log_returns(self):
        self.log_returns = np.log(1+self.data['Adj Close'].pct_change()) 
        self.log_returns = self.log_returns[1:] 

    def volatility_calc(self):
        self.daily_volatility = np.std(self.log_returns) 

    def run_monte_carlo(self):
        self.last_price = self.data['Adj Close'].iloc[-1] 

        all_sims = [] 

        for x in range(self.num_sims):
            price_series = [self.last_price]

            for y in range(1,self.num_days):
                price = price_series[-1]*(1+np.random.normal(0,self.daily_volatility)) 
                price_series.append(price) 
            all_sims.append(price_series) 
        self.simulation_df = pd.DataFrame(all_sims).transpose() 

    def results(self):
        prices = self.simulation_df.iloc[-1] 

        lower_bound_95 = np.percentile(prices,2.5) 
        upper_bound_95 = np.percentile(prices,97.5) 

        mean_price = np.mean(prices) 

        print(f'{self.ticker} expected price for {self.num_days} days later is : {mean_price}')
        print(f'{self.ticker} 95% confidence interval for the price {self.num_days} later is : ({lower_bound_95},{upper_bound_95})') 
        print('\n')

    def plot(self):
        self.simulation_df.plot(legend=False) 
        plt.title(f'Monte Carlo simulation for {self.ticker}') 
        plt.axhline(y=self.last_price, color='black', linestyle='-')
        plt.style.use('seaborn-v0_8-darkgrid')  
        plt.xlabel('Day') 
        plt.ylabel('Price')        

In [11]:
start_date = (datetime.now() - timedelta(days=2*365)).strftime('%Y-%m-%d') # Date two years before from today
end_date = datetime.now().strftime('%Y-%m-%d')
tickerst = ['AES','AMD','AAPL','MSFT','NVDA','AMZN'] 
num_days = 100
num_sims = 1000 

In [12]:
results = {}  # Initialize dictionary to hold results

for tick in tickerst:
    simulation = MonteCarlo(tick,start_date,end_date,num_days,num_sims)
    simulation.fetch_data() 
    simulation.calc_log_returns()
    simulation.volatility_calc()
    simulation.run_monte_carlo()
    simulation.results()  

[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed

AES expected price for 100 days later is : 17.461764846162577
AES 95% confidence interval for the price 100 later is : (11.10268713740198,26.180930291215613)




AMD expected price for 100 days later is : 180.63867362621934
AMD 95% confidence interval for the price 100 later is : (91.39770843532776,315.08356188156927)




[*********************100%%**********************]  1 of 1 completed


...

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

MSFT expected price for 100 days later is : 448.00983732012617
MSFT 95% confidence interval for the price 100 later is : (313.903018302413,603.4015090563771)





[*********************100%%**********************]  1 of 1 completed

NVDA expected price for 100 days later is : 124.57216352335114
NVDA 95% confidence interval for the price 100 later is : (65.44244855884563,213.9435298189781)




AMZN expected price for 100 days later is : 192.5360405098597
AMZN 95% confidence interval for the price 100 later is : (119.78038932628564,287.1906000393314)


